In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-15 12:55:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.84MB/s    in 0.2s    

2022-04-15 12:55:51 (5.84 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://vinekitchendata.s3.amazonaws.com/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [14]:
clean_df = df.dropna()
clean_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [15]:
# Create the vine_table. DataFrame
vine_df = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3DT59XH7HXR9K|          5|            0|          0|   N|                Y|
|R1LFS11BNASSU8|          5|            0|          1|   N|                Y|
|R296RT05AG0AF6|          5|            0|          0|   N|                Y|
|R3V37XDZ7ZCI3L|          5|            0|          1|   N|                Y|
|R14GU232NQFYX2|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [16]:
# Retrieve the rows where the total vote count is greater than 20
from pyspark.sql.functions import col
total_df = vine_df.filter(col("total_votes") >= 20)
total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28RB82UG4RDD5|          5|           20|         20|   N|                Y|
|R3FB6BERWPEIJP|          4|           40|         43|   N|                Y|
|R1D4Z38STRDQXK|          5|           53|         56|   N|                Y|
|R1XMWJZICINIFX|          3|           20|         21|   N|                Y|
|R20QKY1GABXFLM|          1|          272|        297|   N|                Y|
|R3GZ7CK2O1PPG0|          2|           10|         30|   N|                Y|
|R328FA1E6FY3F5|          5|           17|         20|   N|                N|
|R3DH22AA5WGLLS|          5|           30|         30|   N|                N|
|R1E0E5EFZSLJCS|          1|           66|         80|   N|                Y|
|R1TXZQWEHYWEWN|          2|           48|         51|   N|     

In [19]:
# Retrieve the rows where the number of helpful votes divided by the number of total votes is greater than 50%
helpful_df = total_df.withColumn('percent_helpful_votes',col('helpful_votes')/col('total_votes')).alias('percent_helpful_votes').filter(col('percent_helpful_votes') >= 0.5)
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R28RB82UG4RDD5|          5|           20|         20|   N|                Y|                  1.0|
|R3FB6BERWPEIJP|          4|           40|         43|   N|                Y|   0.9302325581395349|
|R1D4Z38STRDQXK|          5|           53|         56|   N|                Y|   0.9464285714285714|
|R1XMWJZICINIFX|          3|           20|         21|   N|                Y|   0.9523809523809523|
|R20QKY1GABXFLM|          1|          272|        297|   N|                Y|   0.9158249158249159|
|R328FA1E6FY3F5|          5|           17|         20|   N|                N|                 0.85|
|R3DH22AA5WGLLS|          5|           30|         30|   N|                N|                  1.0|


In [21]:
# Retrieve the entries where the vine reviews were helpful
helpful_vine_df = helpful_df.filter(col("vine") == 'Y')
helpful_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R1Z71RW4J9IK93|          5|           20|         22|   Y|                N|   0.9090909090909091|
|R3FVB5QI11KI9Q|          4|          192|        200|   Y|                N|                 0.96|
|R2G027YBMVXV6Y|          5|           39|         48|   Y|                N|               0.8125|
|R1QGBAN7BMGWRR|          5|          121|        129|   Y|                N|    0.937984496124031|
|R2NH2AU7XL9ZDZ|          3|           18|         20|   Y|                N|                  0.9|
|R2YVVJ9NOPNX50|          4|           36|         41|   Y|                N|   0.8780487804878049|
|R1XH1LK1FWX3OS|          4|          214|        238|   Y|                N|   0.8991596638655462|


In [22]:
# Retrieve the entries where the non-vine reviews were helpful
helpful_nv_df = helpful_df.filter(col("vine") == 'N')
helpful_nv_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R28RB82UG4RDD5|          5|           20|         20|   N|                Y|                  1.0|
|R3FB6BERWPEIJP|          4|           40|         43|   N|                Y|   0.9302325581395349|
|R1D4Z38STRDQXK|          5|           53|         56|   N|                Y|   0.9464285714285714|
|R1XMWJZICINIFX|          3|           20|         21|   N|                Y|   0.9523809523809523|
|R20QKY1GABXFLM|          1|          272|        297|   N|                Y|   0.9158249158249159|
|R328FA1E6FY3F5|          5|           17|         20|   N|                N|                 0.85|
|R3DH22AA5WGLLS|          5|           30|         30|   N|                N|                  1.0|


In [25]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)
# https://courses.bootcampspot.com/courses/1066/pages/16-dot-4-3-spark-functions?module_item_id=398315
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.GroupedData.agg.html
from pyspark.sql.functions import when,count,lit
ratings_overview_df = helpful_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5_Star_Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("%_5_Star_To_Total"))
ratings_overview_df.show()

+----+-------------+--------------------+------------------+
|vine|Total_Reviews|Total_5_Star_Reviews| %_5_Star_To_Total|
+----+-------------+--------------------+------------------+
|   Y|         1207|                 509| 42.17067108533554|
|   N|        97810|               45846|46.872507923525205|
+----+-------------+--------------------+------------------+

